In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [50]:
dt=pd.read_csv('data.csv')

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import pandas as pd



estimators = [
    ('onehot', OneHotEncoder()),
    ('lr', LogisticRegression())
]
pipe = Pipeline(estimators)

# Just check the encoded variables
ohe = OneHotEncoder()
ohe.fit_transform(dt[['user', 'skill']]).toarray()


# IRT
pipe.fit(dt[['user', 'skill']], dt['correct'])




/opt/homebrew/anaconda3/envs/newenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('onehot', OneHotEncoder()), ('lr', LogisticRegression())])

In [52]:
users=dt.user.unique()
ki=['user_'+str(i) for i in users]
wa=['skill_'+str(j) for j in dt.skill.unique()]

In [53]:
cofs=pipe[1].coef_
diffik=pd.Series(index=dt.skill.unique(),data=cofs.reshape(-1,)[-112:])
scv=diffik.sort_values().index.values
po=[]
for j in dt.skill.values:
    po.append((np.where(scv==j)[0]+1)[0])
dt.skill=po
theta=pd.Series(index=ki,data=cofs.reshape(-1,)[:-112])
diffik=pd.Series(data=diffik.sort_values().values)

fiffik=diffik-diffik.min()
def sigmoid(x):
    return 1/(1+np.exp(-x))
def update_theta(theta,action,K):
    probs=sigmoid(K*(theta-action))
    haz=np.random.binomial(1,p=probs)
    return theta+K*(haz-probs)


In [54]:

def upda_thet(theta,action,K,out):
    probs=sigmoid(K*(theta-action))
    
    return theta+K*(out-probs)

In [55]:
wa={}
for i in np.arange(-10.8,10.1,.2):
    wa[np.around(i,decimals=1)]=np.around(i,decimals=1)
for j in np.arange(-10.9,10,0.2):
    wa[np.around(j,decimals=1)]=np.around(j-0.1,decimals=1)

In [56]:
fake_dic_panda_real={}
for j in dt.user.unique():
    m=dt[dt.user==j]
    fake_dic_panda_real['user_'+str(j)]=pd.DataFrame(index=np.arange(len(m)))
    fake_dic_panda_real['user_'+str(j)]['theta_'+str(j)]=0
    fake_dic_panda_real['user_'+str(j)]['action_'+str(j)]=m.skill.values
    l=[0]
    r=[]
    for i in range(len(m)):
        l.append(wa[np.around(upda_thet(l[-1],diffik.iloc[m.skill.iloc[i]-1],1,m.correct.iloc[i]),decimals=1)])
        r.append(((l[-1]-l[-2])>0)*fiffik.iloc[m.skill.iloc[i]-1])
    fake_dic_panda_real['user_'+str(j)]['theta_'+str(j)]=l[:-1]
    fake_dic_panda_real['user_'+str(j)]['reward_'+str(j)]=r
    fake_dic_panda_real['user_'+str(j)]['new_theta_'+str(j)]=l[1:]

In [57]:
fake_dic_panda={}
for j in dt.user.unique():
    m=dt[dt.user==j]
    fake_dic_panda['user_'+str(j)]=pd.DataFrame(index=np.arange(len(m)))
    fake_dic_panda['user_'+str(j)]['theta_'+str(j)]=0
    fake_dic_panda['user_'+str(j)]['action_'+str(j)]=m.skill.values
    l=[0]
    r=[]
    for i in fake_dic_panda['user_'+str(j)]['action_'+str(j)]:
        l.append(wa[np.around(update_theta(l[-1],diffik.iloc[i-1],1),decimals=1)])
        r.append(((l[-1]-l[-2])>0)*fiffik.iloc[i-1])
    fake_dic_panda['user_'+str(j)]['theta_'+str(j)]=l[:-1]
    fake_dic_panda['user_'+str(j)]['reward_'+str(j)]=r
    fake_dic_panda['user_'+str(j)]['new_theta_'+str(j)]=l[1:]

In [130]:
a=dt.groupby(['user','skill']).size()

In [132]:
dt[dt.user==0]

,user,item,skill,correct,wins,fails
10540,0,54003,35,0,0,0
10557,0,53991,35,1,0,1
10567,0,54071,35,0,1,1
10588,0,54015,35,1,1,2
10608,0,53987,35,1,2,2
...,...,...,...,...,...,...
190611,0,59762,6,0,6,5
190612,0,59763,6,1,6,6
190613,0,59764,6,1,7,6
190614,0,59765,6,1,8,6
